In [1]:
import cv2
import math
import numpy as np
import skimage.io as io
from scipy.ndimage import find_objects
from skimage.feature import canny
import matplotlib.pyplot as plt
from skimage.color import rgb2gray , rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.filters import threshold_otsu, threshold_mean, threshold_li, threshold_isodata, threshold_niblack , median
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin , disk
from skimage.draw import rectangle, rectangle_perimeter
from skimage.transform import resize
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
def multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame):
    
    # Extracting the blue, red, green and alpha channels
    
    B = rgba_frame[:,:,0]
    G = rgba_frame[:,:,1]
    R = rgba_frame[:,:,2]
    A = rgba_frame[:,:,3]
    
    #Extracting the Hue, Saturation and vue channels
    
    H = hsv_frame[:,:,0]
    S = hsv_frame[:,:,1]
    V = hsv_frame[:,:,2]
    
    # Extracting the Y, Cr and Cb channels
    
    Y = ycbcr_frame[:,:,0]
    Cr = ycbcr_frame[:,:,1]
    Cb = ycbcr_frame[:,:,2]
    
    #     Applying  Thresholding using Log-Chromaticity color space 
    n_r = np.ma.divide(R, G)
    n_b = np.ma.divide(B, G)
    log_rg = np.ma.log( n_r )
    log_bg = np.ma.log( n_b )
    condition_1 = (log_rg>=0.15) & (log_rg<=1.1)
    condition_2 = (log_bg>=-4) & (log_bg<=0.3)
    mask_1 = condition_1 & condition_2
    
    # Extracting masks based on a combination of RGBA, HSV and YCrCb models for skin detection
    
    mask_rgb = (R>95)&(G>40)&(B>20)&(R>G)&(R>B)&(abs(R-G)>15)&(A>15)
    mask_Ycbcr = (Cr > 135)&(Cb>85)&(Y>80)&(Cr <= (1.5862*Cb)+20)&(Cr>=(0.3448*Cb)+76.2069)&(Cr >= (-4.5652*Cb)+234.5652)&(
                  Cr <= (-1.15*Cb)+301.75)&(Cr <= (-2.2857*Cb)+432.85)
    mask_hsv = ((0.0 <= H) & (H <= 50.0))&((0.23 <= S/255) & (S/255 <= 0.68))

    mask1 = mask_rgb & mask_Ycbcr
    mask2 = mask_rgb & mask_hsv
    
    return mask_1


In [3]:
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [4]:
def moving_area(image1 , image2 , image3):
    diff1 = cv2.absdiff(currentFrameGray,lastFrameGray)
    diff2 = cv2.absdiff(currentFrameGray,beforeLastFrameGray)    
    _, binary1 = cv2.threshold(diff1,threshold,1,cv2.THRESH_BINARY)
    _, binary2 = cv2.threshold(diff2,threshold,1,cv2.THRESH_BINARY)

    resultImageDiff = cv2.bitwise_and(binary1, binary2)
    resultImageDiff = cv2.erode(resultImageDiff, np.ones((5, 5),dtype='uint8'), iterations = 2)

    ILog.d(resultImageDiff, 'imgdiff', preFunc=(multiplyImage(255),))

    objs = find_objects(resultImageDiff)

    if not objs:
        mROI=[0,0,0,0]
    else:        
        mROI=[objs[0][0].start,objs[0][0].stop,objs[0][1].start,objs[0][1].stop]
    
    return resultImageDiff, mROI
    

In [5]:
def classify(mask, frame):
    try:  #an error comes if it does not find anything in window as it cannot find contour of max area
              #therefore this try error statement
            
        ## code starts here
        #temp = np.copy(mask).astype("uint8")*255
        mask = mask.astype("uint8")*255
        
    #find contours
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #find bouding box
        Xmax = max(cnt[:,0,0])
        Ymax = max(cnt[:,0,1])
        Xmin = min(cnt[:,0,0])
        Ymin = min(cnt[:,0,1])

        box = [Xmin, Ymin, Xmax, Ymax]

        boxWidth = Xmax - Xmin
        boxHeight = Ymax - Ymin

        boxarea = boxHeight * boxWidth

        boxAspectRatio = boxHeight / boxWidth

    #find orientation
        orientation = 'V'
        if(boxAspectRatio <= 1):
            orientation = 'H'
            
    #left half
        leftHalfCnt = cnt[(cnt[:,0,0] <= box[0]+boxWidth/2)]
        leftHalfCntArea = cv2.contourArea(leftHalfCnt)
    
    #right half
        rightHalfCnt = cnt[(cnt[:,0,0] > box[0]+boxWidth/2)]
        rightHalfCntArea = cv2.contourArea(rightHalfCnt)
    
    
    #resizing contour
        windowCenter = (Xmin + boxWidth // 2, Ymin + boxHeight // 2)
        windowSide = 0
        if(boxHeight > boxWidth):
            windowSide = boxHeight
        else:
            windowSide = boxWidth
        
        
        window = frame[windowCenter[0] - windowSide // 2 : windowCenter[0] + windowSide // 2,
                      windowCenter[1] - windowSide // 2 : windowCenter[1] + windowSide // 2]
#         window = resize(window, (128, 128))
        roi = window
        
#     #top third
#         Xmin = min(cnt[:,0,0][(cnt[:,0,0] >= box[0]) & (cnt[:,0,0] <= box[0]+boxWidth/3)])
#         Ymin = min(cnt[:,0,1][(cnt[:,0,0] >= box[0]) & (cnt[:,0,0] <= box[0]+boxWidth/3)])
#         Xmax = max(cnt[:,0,0][(cnt[:,0,0] >= box[0]) & (cnt[:,0,0] <= box[0]+boxWidth/3)])
#         Ymax = max(cnt[:,0,1][(cnt[:,0,0] >= box[0]) & (cnt[:,0,0] <= box[0]+boxWidth/3)])

#         topThirdBox = [Xmin, Ymin, Xmax, Ymax]

#         topThirdWidth = Xmax - Xmin

#         topThirdWidthRatio = topThirdWidth / boxWidth

#     # most left 
#         Xmin = min(cnt[:,0,0][(cnt[:,0,1] >= box[1]) & (cnt[:,0,1] <= box[1]+boxHeight/3)])
#         Ymin = min(cnt[:,0,1][(cnt[:,0,1] >= box[1]) & (cnt[:,0,1] <= box[1]+boxHeight/3)])
#         Xmax = max(cnt[:,0,0][(cnt[:,0,1] >= box[1]) & (cnt[:,0,1] <= box[1]+boxHeight/3)])
#         Ymax = max(cnt[:,0,1][(cnt[:,0,1] >= box[1]) & (cnt[:,0,1] <= box[1]+boxHeight/3)])

#         mostLeftBox = [Xmin, Ymin, Xmax, Ymax]

#         mostLeftHeight = Ymax - Ymin

#         mostLeftHeightRatio = mostLeftHeight / boxHeight


    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)


    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
    #vertical Fingers Lines
        vLines = []
        vPoints = []
        vAnchor = np.array([(Xmax + Xmin) / 2, Ymax])
        
#         if(count % 100 == 0):
        lastPoint = hull[0,:,:]
        for i in hull:
#             a = math.sqrt((lastPoint[0,0] - i[0,0])**2 + (lastPoint[0,1] - i[0,1])**2)
#             b = math.sqrt((vAnchor[0] - i[0,0])**2 + (vAnchor[1] - i[0,1])**2)
#             c = math.sqrt((lastPoint[0,0] - vAnchor[0])**2 + (lastPoint[0,1] - vAnchor[1])**2)
#             s = (a + b + c) / 2
#             ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
#             # distance 
#             if(a != 0):
#                 d=(2*ar)/a
#             else:
#                 d = 0;
                
#             # apply cosine rule here
#             if(2*b*c != 0):
#                 angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
#             else:
#                 angle = 0
#                 print(i[0,:])
#                 print(vAnchor)
            if(dist(i[0,:], vAnchor) > 80 and np.absolute(i[0,0] - lastPoint[0,0]) > 10):
                
#                 if(count % 100 == 0): print(dist(i[0,:],vAnchor));
#                 print(lastPoint[0,0] - i[0,0])
#                 cv2.circle(roi, (i[0, 0], i[0, 1]), 3, [255,255,255], -1)
                vPoints.append((i[0, 0], i[0, 1]))
                vLines.append([(int(i[0,0]),int(i[0,1])), (int(vAnchor[0]), int(vAnchor[1]))])
                lastPoint = i
        
       
    #horizontal fingers lines
        hLines = []
        hPoints = []
        hAnchor = np.array([Xmax, (Ymax + Ymin) / 2])
                           
#         if(count % 100 == 0):
        lastPoint = hull[0,:,:]
        for i in hull:
#                 print(i[0,:])
#                 print(vAnchor)
            if(dist(i[0,:],hAnchor) > 80 and dist(i[0,:], lastPoint[0,:]) > 10):        
#                 if(count % 100 == 0): print(dist(i[0,:],vAnchor));
#                 print(lastPoint[0,0] - i[0,0])
#                 cv2.circle(roi, (i[0, 0], i[0, 1]), 3, [255,255,255], -1)
                hPoints.append((i[0, 0], i[0, 1]))
                hLines.append([(int(i[0,0]),int(i[0,1])), (int(hAnchor[0]), int(hAnchor[1]))])
                lastPoint = i
   
    # drawing fingers lines depending on orientation
#         if(orientation == 'V'):
#             for i in vLines:
#                 cv2.line(roi, i[0], i[1], [0,255,0], 2)
#             for i in vPoints:
#                 cv2.circle(roi, i, 3, [255,255,255], -1)
#         else:
#             for i in hLines:
#                 cv2.line(roi, i[0], i[1], [0,0,255], 2)
#             for i in hPoints:
#                 cv2.circle(roi, i, 3, [255,255,255], -1)

        
     #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)

    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100

     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)

        contourRatio = areacnt / boxarea

    # no. of defects in each direction
        upFingers = 0
        downFingers = 0
        leftFingers = 0
        leftFingersRight = 0
        rightFingersAcute = 0
        rightFingersRight = 0

    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)

            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

            #distance between point and convex hull
            d=(2*ar)/a

            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

            if (far[1] > start[1]) & (far[1] > end[1]) & (far[0] < start[0]) & (far[0] > end[0]): 
                if (angle <= 70 and d > 30):
                    upFingers += 1
                    cv2.circle(roi, far, 3, [255,0,0], -1)
            elif (far[0] > start[0]) & (far[0] > end[0]) & (far[1] > start[1]) & (far[1] < end[1]):
                if (angle <= 50 and d > 30):
                    leftFingers += 1
                    cv2.circle(roi, far, 3, [0,0,0], -1)
                elif (angle >= 50 and angle <=120 and d > 30):
                    leftFingersRight += 1
                    cv2.circle(roi, far, 3, [100,100,100], -1)
            elif (far[1] < start[1]) & (far[1] < end[1]) & (far[0] > start[0]) & (far[0] < end[0]):
                if (angle <= 70 and d > 30):
                    downFingers += 1
                    cv2.circle(roi, far, 3, [0,255,0], -1)
            elif (far[0] < start[0]) & (far[0] < end[0]):
                if (angle >= 70 and angle <= 120 and d > 30):
                    rightFingersRight += 1
                    cv2.circle(roi, far, 3, [0,0,255], -1)
                elif (angle <= 70 and d > 30):
                    rightFingersAcute += 1
                    cv2.circle(roi, far, 3, [255, 0, 255], -1)

            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)

        upFingers += 1
        downFingers += 1
        leftFingers += 1
        leftFingersRight += 1
        rightFingersAcute += 1
        rightFingersRight += 1
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        currLetter = ''
        if upFingers == 1 and downFingers == 1 and leftFingers == 1 and rightFingersAcute == 1 and rightFingersRight == 1:
            if areacnt<2000:
                 currLetter = 'Put hand in the box'
            else:
                if arearatio < 8.5: 
                    # no fingers
                    if boxAspectRatio < 1.5:
                        currLetter = 'A'
                    else:
                        currLetter = 'B'
                else: 
                    # one finger
                    if orientation == 'V':
                        if leftHalfCntArea < rightHalfCntArea:
                            currLetter = 'D'
                        else:
                            currLetter = 'I'
                    elif orientation == 'H' and leftHalfCntArea < rightHalfCntArea:
                        currLetter = 'G'
#                 if len(vLines) == 0:
#                     currLetter ='A'
#                 elif len(vLines) == 1:
#                     if leftHalfCntArea < rightHalfCntArea:
#                         currLetter = 'D'
#                     else:
#                         currLetter = 'I'
#                 elif len(vLines) == 2:
#                     currLetter ='U'
#                 elif len(vLines) == 3:
#                     currLetter ='F'
#                 elif len(vLines) == 4:
#                     currLetter ='B'
                 
        elif upFingers == 2 and (rightFingersRight == 2 or rightFingersAcute == 2):
            currLetter = '3'
        
        elif upFingers == 2:
            currLetter = 'V'
        
        elif (upFingers == 4 and (rightFingersAcute == 2 or rightFingersRight == 2)) or (upFingers == 3 and ((rightFingersAcute ==2 and rightFingersRight == 2) or rightFingersAcute == 3)):
            currLetter = '5'
        
        elif upFingers == 4 or (upFingers == 3 and rightFingersAcute == 2):
            currLetter = '4'
        
        elif upFingers == 3:
            currLetter = 'W'
           
        elif rightFingersAcute == 2:
            currLetter = 'C'
            
        elif rightFingersRight == 2:
            currLetter = 'L'
            
        elif leftFingers == 3:
            currLetter = 'E'
        
        elif leftFingersRight == 2:
            currLetter = 'P'
        
        elif downFingers == 2:
            currLetter = 'N'
        
        elif downFingers == 3:
            currLetter = 'M'
        
        else:
            currLetter = 'Reposition'

        cv2.putText(frame,currLetter,(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    
        
        # show the windows
#         cv2.imshow('mask', mask)
        cv2.imshow('frame', frame)
    
    except Exception as e:
        print(e)
   

In [6]:
cap = cv2.VideoCapture(0)


# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

ok_region = False
old_moving_region = -1 

_,frame1 = cap.read()
# converting the image into grayscale image
frame1 = cv2.resize(frame1, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
image3 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
backgroundmodel = image3
_,frame2 = cap.read()
# converting the image into grayscale image
frame2 = cv2.resize(frame2, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
image2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

fgbg =cv2.createBackgroundSubtractorKNN(history=1000) #,detectShadows=True)


while True:
    
    ret, rgb_frame = cap.read()
    
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    rgb_frame = cv2.GaussianBlur(rgb_frame ,(7,7),cv2.BORDER_DEFAULT) 
    image1 = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
     
    rgb_frame=cv2.flip(rgb_frame,1)
#     backgroundmodel,fgmask = moving_area(image1 , image2 , image3 ,0.95,backgroundmodel,30)
    
    fgmask = fgbg.apply(rgb_frame)
    dilated_fgmask = binary_dilation(fgmask ,disk(3))
    fgmask = np.zeros(image1.shape)
    fgmask[dilated_fgmask] = 1
    
    cv2.imshow('frame',fgmask)
    
    if(not ok_region):
        ok_region = True
        old_moving_region = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
        
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
    gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces: 
#         To draw a rectangle in a face  
        cv2.rectangle(rgb_frame,(x,y),(x+w,y+h),(255,255,0),2)   
     
    
    # converting the rgb space to hsv space color
    hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
    # converting the rgb space to YCbCr space color
    ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
    # converintg the rgb space to rgba spave 
    rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
    
#     moving_region = moving_area(image1 , image2 , image3)
#     if(np.sum(moving_region) <= 10000):
#         moving_region = old_moving_region
#     old_moving_region = moving_region
    
    # the final mask to extract the binary image
    mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    binary_frame2[mask] = 1
  
    for (x,y,w,h) in faces: 
        binary_frame2[y-30:y+h+30 , x:x+w] = 0
        
    #removing noise
    eroded_frame = cv2.erode(binary_frame2, np.ones((3,3)), iterations = 1)
    dilated_frame = cv2.dilate(eroded_frame, np.ones((10,5)), iterations = 1)
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    temp = np.copy(binary_frame2).astype("uint8")
    
    binary_frame2[dilated_frame>=1] = 1
    temp[dilated_frame>=1] = 255
    
    contours, hierarchy = cv2.findContours(temp, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    bounding_boxes = list()
#     mask_face = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     for contour in contours:
#         x,y,w,h = cv2.boundingRect(contour)
#         if( ( (float(w)/h <= 0.9 and float(h)/w >= 1.3) or (float(h)/w <= 0.5 and float(w)/h >= 1.3 ) )):
# #         if( ( (float(w)/h <= 0.9) or (float(h)/w <= 0.5 and float(w)/h >= 1.3 ) )):
#             mask_face[y:y+h , x:x+w] = 1
#             cv2.drawContours(rgb_frame, contour, -1, (0, 255, 0), 3)
    
    #cv2.imshow('binary canny detection', final_frame)
#     cv2.imshow('canny detection', cannyImg )

    frame_with_hand_only = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    frame_with_hand_only[(binary_frame2>=1) &  (fgmask >= 1)] = 1
    
    cv2.imshow('Input', rgb_frame)
    cv2.imshow('freame', fgmask)
    cv2.imshow('Final Output', frame_with_hand_only)
    classify(frame_with_hand_only, rgb_frame)
    #cv2.imshow('moving region',moving_region)
    image3 = image2
    image2 = image1
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()
